In [2]:
import torch
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# ✅ Set device manually
device = 0 if torch.cuda.is_available() else -1

# ✅ Workaround for torch internal dtype bug
pipe = pipeline("text2text-generation", model="google/flan-t5-large", device=device, max_length=100)


llm = HuggingFacePipeline(pipeline=pipe)

# ✅ LangGraph node
def question_node(state: dict) -> dict:
    question = state["input"]
    answer = llm.invoke(question)
    return {"output": answer}

# ✅ Build LangGraph
builder = StateGraph(dict)
builder.add_node("question_node", RunnableLambda(question_node))
builder.set_entry_point("question_node")
builder.add_edge("question_node", END)

graph = builder.compile()

# ✅ Run
response = graph.invoke({"input": "Who is the current Prime Minister of India?"})
print("Answer:", response["output"])


Device set to use cpu


Answer: narendra modi
